In [0]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages
Requirement already up-to-date: protobuf>=3.4.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: termcolor>=1.1.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: tensorboard<1.8.0,>=1.7.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: gast>=0.2.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: numpy>=1.13.3 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: grpcio>=1.8.6 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: six>=1.10.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: wheel>=0.26 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: absl-py>=0.1.6 in /usr/l

In [0]:
from urllib.request import urlretrieve
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from zipfile import ZipFile
import math

import tensorflow as tf
tf.enable_eager_execution()
import tensorflow.contrib.eager as tfe


def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

download('https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip','data.zip')
print("All the files are downloaded")

Instructions for updating:
Use the retry module or similar alternatives.
All the files are downloaded


In [0]:
def uncompress_features_labels(dir):
    if(os.path.isdir('data')):
        print('Data extracted')
    else:
        with ZipFile(dir) as zipf:
            zipf.extractall('data')
uncompress_features_labels('data.zip')

def data_Files(mypath):
    onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
    print(onlyfiles)

data_Files('data')

Data extracted
['train.p', 'valid.p', 'test.p']


In [0]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'data/train.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

#Flaaten the Images data so that it can be a 1-D array
X_train, y_train = train['features'].flatten().reshape(len( train['features']), 1, 3072), train['labels']
X_test, y_test = test['features'].flatten().reshape(len( test['features']), 1, 3072), test['labels']
print('Data Loaded Successfully')

Data Loaded Successfully


In [0]:
#Convert data into float
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

#Convert data into float and take one hot of the output classes
y_train = y_train.astype(np.float32) 

y_train = tf.one_hot(y_train, depth = 43)

y_test = tf.one_hot(y_test.astype(np.float32), depth = 43)

In [0]:
def data(features, labels, batch_size):

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(((features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(100).batch(batch_size)

    return dataset

In [0]:
dataset = data(X_train, y_train, 128)
#get the batch from dataset.
features, label = tfe.Iterator(dataset).next()
#Print the features from data. Normally this would return a tensor object but with eager execution you can check the values
print(features)

tf.Tensor(
[[[232. 207. 201. ... 122. 119. 105.]]

 [[ 36.  31.  28. ...  22.  19.  17.]]

 [[ 80.  91.  98. ...  82.  54.  38.]]

 ...

 [[240. 220. 197. ... 107. 112.  93.]]

 [[ 67.  82.  96. ...  44.  33.  24.]]

 [[ 46.  36.  19. ...  19.  17.  14.]]], shape=(128, 1, 3072), dtype=float32)


In [0]:
#Declare the model. Note: I know this is not the best model out there:)
model = tf.keras.Sequential([
  tf.keras.layers.Dense(200, activation="tanh", input_shape=(3072,)),  # input shape required
  tf.keras.layers.Dense(100, activation="tanh"),
  tf.keras.layers.Dense(43, activation='softmax')
])

In [0]:
#Declare optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
#Define loss function
def loss(model, x, y):
  y_ = model(x)
  return tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_)
#Compute gradient
def grad(model, inputs, targets):
  with tfe.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, model.variables)

In [0]:
train_loss_results = []
train_accuracy_results = []

num_epochs = 5
for epoch in range(num_epochs):
  epoch_loss_avg = tfe.metrics.Mean()
  epoch_accuracy = tfe.metrics.Accuracy()
  i = 0
  # Training loop - using batches of 32
  for x, y in tfe.Iterator(dataset):
    # Optimize the model
    grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())
    # Track progress
    epoch_loss_avg(loss(model, x, y))  # add current batch loss
  correct_pred = tf.equal(tf.cast(tf.argmax(model(X_test), axis=1), tf.float32), y_test)
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

  # end epoch
  
  print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3f}".format(epoch, epoch_loss_avg.result(), accuracy))

Epoch 000: Loss: 3.742, Accuracy: 0.977
Epoch 001: Loss: 3.743, Accuracy: 0.977
Epoch 002: Loss: 3.739, Accuracy: 0.977
Epoch 003: Loss: 3.748, Accuracy: 0.977
Epoch 004: Loss: 3.711, Accuracy: 0.977
